<a href="https://colab.research.google.com/github/kaipak/from-eyes-to-ears/blob/slitt987/From_Eyes_to_Ears.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup the Environment

In [2]:
!git clone https://github.com/kaipak/from-eyes-to-ears.git
%cd from-eyes-to-ears
!git checkout slitt987

fatal: destination path 'from-eyes-to-ears' already exists and is not an empty directory.
/content/from-eyes-to-ears
Already on 'slitt987'
Your branch is up to date with 'origin/slitt987'.


In [3]:
!git pull

Already up to date.


In [4]:
!pip install -f https://download.pytorch.org/whl/cu101/torch_stable.html -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html -r requirements.txt

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html, https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-8ko2emfn
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-8ko2emfn
     |████████████████████████████████| 753.4MB 22kB/s 
     |████████████████████████████████| 4.0MB 55.5MB/s 
     |████████████████████████████████| 276kB 62.1MB/s 
     |████████████████████████████████| 6.2MB 1.9MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44074 sha256=27c95c6b0226c7e0cb25eb11ee3f4a3c519a5ac17e59ce84faf4f5cd108f7aa1
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=275268 sha256=4d72908910f4dad09d2db429ee42b161c9c9c2f90de6e9625e1fad499361dd15
  Stored in d

In [0]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import torch, torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# import files module
from google.colab import files

## Model Time

### First let's setup our Panoptic Segmentation Model
We are going to use this trained out of the box because re-training would take days and a ton of horsepower.

In [0]:
po_cfg = get_cfg()
po_cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
po_cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
predictor = DefaultPredictor(po_cfg)

Cool, but that model only has 80 thing and 54 stuff categories.  We want to add another custom group of things, but honestly we don't want to re-train everything.  So instead we will build a seperate mask-r-cnn model (a bit simpler and easier to find training data for) to learn these new things.

We will use a masked model so that we can overlay the results of these two models together at a pixel level.

In [0]:
# download, decompress the data
!wget https://github.com/matterport/Mask_RCNN/releases/download/v2.1/balloon_dataset.zip
!unzip balloon_dataset.zip > /dev/null

In [12]:
print("Provide a test image")
files.upload()

Provide a test image


MessageError: ignored

In [11]:
!ls

detectron2_tutorial.ipynb  README.md  requirements.txt
